# Actividad de laboratorio #

1. Dado el siguiente código:

In [0]:
#include <stdio.h>
#include <stdlib.h>
#include <unistd.h>
#include<time.h>

long long int factorial (int);

int main (int argc, char *argv[]) {
  srand(time(0)); 
  if (argc== 2) {
    printf ("El factorial de %s es %lld\n", argv[1], factorial (atoi (argv[1])));
  }
  return 0;
}

long long int factorial (int n) {
  long long int resultado= 1;
  int num;
  for (num= 2; num<= n; num++) {
    resultado= resultado*num;
    printf ("Factorial de %d, resultado parcial %lld\n", n, resultado);
    sleep (random()%3);
  }
  return resultado;
}

Modifique el programa anterior de manera que reciba dos números enteros como parámetros de entrada y calcule sus factoriales de manera concurrente utilizando dos hilos que se ejecutan en paralelo con el hilo principal. El hilo principal deberá esperar a que terminen los otros dos hilos. Recuerda  añadir el parámetro ```-lpthread``` al comienzo del ```gcc``` para compilar la aplicación multihilo.

2. Modifique el programa resultante del numeral anterior de manera que no el número de factoriales a calcular no este limitado a 2. Para este caso, el programa puede crear tantos hilos como sean necesarios para el cálculo del factorial dependiendo del número de argumentos pasados. El hilo principal deberá esperar a que terminen el resto de los hilos y a medida que vayan terminando muestran un mensaje que indique el identificador del hilo finalizado.



3. El siguiente programa cuenta el número de veces que el carácter a o A aparece en el fichero indicado como parámetro de entrada. Modifíquese para que ahora se cree un hilo y sea este el que ejecute la función cuenta.

In [0]:
#include <unistd.h>
#include <stdlib.h>
#include <stdio.h>
#include <sys/types.h>
#include <sys/stat.h>
#include <fcntl.h>

#define MAXLON 1000

void cuenta (char *);

int main (int argc, char *argv[]) { 
  if (argc!= 2) {
    printf ("Indica el nombre de un fichero.\n");
    exit(0);
  }
  cuenta(argv[1]);
  return 0;
}

void cuenta (char *nombre) {
  int pos, cont= 0, leidos;
  char cadena[MAXLON];
  int fd;
  fd= open(nombre,O_RDONLY);
  while ((leidos =read(fd,cadena,MAXLON))!= 0) {
    for (pos= 0; pos< leidos; pos++) {
      if ((cadena[pos]== 'a') || (cadena[pos]== 'A')) {
        cont++;
      }
    }
  }
  printf("Fichero %s: %d caracteres 'a' o 'A' encontrados\n", nombre, cont);
  close(fd);
}

Codigo agragando un hilo que lance la funciòn 

In [0]:
#include <unistd.h>
#include <stdlib.h>
#include <stdio.h>
#include <sys/types.h>
#include <sys/stat.h>
#include <fcntl.h>
#include <pthread.h>

#define MAXLON 1000

struct cuenta_parms
{
  /* The character to print.  */
  char * nombre;

};

void * cuenta (void* parameters) {
  struct cuenta_parms* p = (struct cuenta_parms*) parameters;

  int pos, cont= 0, leidos;
  char cadena[MAXLON];
  int fd;
  fd= open(p->nombre,O_RDONLY);
  while ((leidos =read(fd,cadena,MAXLON))!= 0) {
    for (pos= 0; pos< leidos; pos++) {
      if ((cadena[pos]== 'a') || (cadena[pos]== 'A')) {
        cont++;
      }
    }
  }
  printf("Fichero %s: %d caracteres 'a' o 'A' encontrados\n", p->nombre, cont);
  close(fd);
  return NULL;
}

int main (int argc, char *argv[]) {
  if (argc!= 2) {
    printf ("Indica el nombre de un fichero.\n");
    exit(0);
  }

  pthread_t thread1_id;
  struct cuenta_parms thread1_args;

  thread1_args.nombre = argv[1];

  pthread_create (&thread1_id, NULL, &cuenta, &thread1_args);

  /*-------INSERTAR AQUÍ-------*/
  pthread_join (thread1_id, NULL);
 
  return 0;
}

Resultados

![texto alternativo](https://raw.githubusercontent.com/josearangos/thread-api/master/Code/Punto%203/P_3.png)

4. Modifique el programa resultado del ejercicio anterior para que se creen tantos hilos como ficheros especificados como parámetros de entrada de tal manera que todos los hilos lleven a cabo su función de forma concurrente.

In [0]:
#include <unistd.h>
#include <stdlib.h>
#include <stdio.h>
#include <sys/types.h>
#include <sys/stat.h>
#include <fcntl.h>
#include <pthread.h>

#define MAXLON 1000

struct cuenta_parms
{
  /* The character to print.  */
  char * nombre;

};

void * cuenta (void* parameters) {
  struct cuenta_parms* p = (struct cuenta_parms*) parameters;

  int pos, cont= 0, leidos;
  char cadena[MAXLON];
  int fd;
  fd= open(p->nombre,O_RDONLY);
  while ((leidos =read(fd,cadena,MAXLON))!= 0) {
    for (pos= 0; pos< leidos; pos++) {
      if ((cadena[pos]== 'a') || (cadena[pos]== 'A')) {
        cont++;
      }
    }
  }
  printf("Fichero %s: %d caracteres 'a' o 'A' encontrados\n", p->nombre, cont);
  close(fd);
  return NULL;
}

int main (int argc, char *argv[]) {
  if (argc < 2) {
    printf ("Indica el nombre de un fichero.\n");
    exit(0);
  }
  int count=1;
  while(count<argc){

    pthread_t thread1_id;
    struct cuenta_parms thread1_args;

    thread1_args.nombre = argv[count];

    pthread_create (&thread1_id, NULL, &cuenta, &thread1_args);

    /*-------INSERTAR AQUÍ-------*/
    pthread_join (thread1_id, NULL);

    count+=1;
  }
  return 0;
}


Resultados

![texto alternativo](https://raw.githubusercontent.com/josearangos/thread-api/master/Code/Punto%204/P_4.png)

5. Se requiere un programa que reciba un vector de números a través de un archivo de texto. La idea es que el programa sume todos los números del vector. Implemente el programa de dos maneras: la primera de una forma estrictamente secuencial. La segunda forma es creando dos hilos, de manera que cada uno de ellos realice la sumatoria de la mitad de los componentes del vector. El hilo 1 sumará los primeros datos del vector y el hilo 2 los últimos. Luego cuando los dos hilos finalicen muestre en pantalla el resultado.
   * Realice el programa de manera genérica, de tal forma que sea posible ingresar vectores de cualquier tamaño.
   * Mida el tiempo de ejecución de ambas implementaciones para varios tamaños del vector 
   *  ¿El resultado obtenido es acorde a lo que usted esperaba?
   Describa la técnica que usó para realizar la medición del tiempo. ¿Cuáles son las debilidades de esta técnica? ¿Existe otra forma de medir el tiempo de ejecución de un programa?

 Programa que suma todos los números de un vector

In [0]:
#include <unistd.h>
#include <stdlib.h>
#include <stdio.h>
#include <sys/types.h>
#include <sys/stat.h>
#include <fcntl.h>
#include <string.h>

#define MAXLON 1000

void cuenta (char *);

int main (int argc, char *argv[]) { 
  if (argc!= 2) {
    printf ("Indica el nombre de un fichero.\n");
    exit(0);
  }
  cuenta(argv[1]);
  return 0;
}

void cuenta (char *nombre) {
  int pos, cont= 0, leidos;
  char cadena[MAXLON];
  int fd;
  fd= open(nombre,O_RDONLY);
  char valores [7];
  int suma=0;
  while ((leidos =read(fd,cadena,MAXLON))!= 0) {
    for (pos= 0; pos< leidos; pos++) { 
      if (cadena[pos]== ','){
        //valores[cont]=NULL;
        suma=suma+atoi(valores);
        memset(valores,'\0',7);
        cont=0;
      }
      else{
        valores[cont]=cadena[pos];
        cont++;
      }
    }
  }
  suma=suma+atoi(valores);
  printf("Fichero %s:la suma de los valores encontraods es: %d \n", nombre, suma);
  close(fd);
}

Resultados

![texto alternativo](https://raw.githubusercontent.com/josearangos/thread-api/master/Code/Punto%205/P_5.1.png)

Segunda forma  creando dos hilos, de manera que cada uno de ellos realice la sumatoria de la mitad de los componentes del vector

In [0]:
#include <unistd.h>
#include <stdlib.h>
#include <stdio.h>
#include <sys/types.h>
#include <sys/stat.h>
#include <fcntl.h>
#include <string.h>
#include <pthread.h>

#define MAXLON 1000

int result=0;

struct cuenta_parms
{
  /* The character to print.  */
  char * nombre;
  int punteroa;
  int punterod;
};
int cuentaCaracter (char *nombre) {
  int pos, cont= 0, leidos;
  char cadena[MAXLON];
  int fd;
  fd= open(nombre,O_RDONLY);
  while ((leidos =read(fd,cadena,MAXLON))!= 0) {
    for (pos= 0; pos< leidos; pos++) {
      if (cadena[pos]== ','){
        cont++;
      }
    }
  }
  cont=cont+1;
  int  div=((cont%2) ? cont/2+1:cont/2);
  printf("El valor de la división es %d ",div);
  cont= 0;
  close(fd);
  fd= open(nombre,O_RDONLY);
  while ((leidos =read(fd,cadena,MAXLON))!= 0) {
    for (pos= 0; pos< leidos; pos++) {
      if (cadena[pos]== ','){
        cont++;
        if(cont==div){
          return pos+1;
        }
      }
    }
  }
  close(fd);
  return 0;
}
void *cuenta (void* parameters) {
  struct cuenta_parms* p = (struct cuenta_parms*) parameters;
  int pos, cont= 0, leidos;
  char cadena[MAXLON];
  int fd;
  fd= open(p->nombre,O_RDONLY);
  char valores [7];
  int suma=0;
  int hasta=p->punteroa;
  while ((leidos =read(fd,cadena,MAXLON))!= 0) {
    if(p->punteroa==-1){
     hasta=leidos;
    }
    for (pos=p->punterod; pos< hasta; pos++) { 
      if (cadena[pos]== ','){
        
        suma=suma+atoi(valores);
        memset(valores,'\0',7);
        cont=0;
      }
      else{
        valores[cont]=cadena[pos];
        cont++;
      }
    }
  }
  suma=suma+atoi(valores);
  result=result+suma;
  printf("Fichero %s: la suma de los valores parciales es: %d \n", p->nombre, suma);
  close(fd);
  return NULL;
}
int main (int argc, char *argv[]) { 

  if (argc!= 2) {
    printf ("Indica el nombre de un fichero.\n");
    exit(0);
  }
  int cvalores=cuentaCaracter (argv[1]);

  pthread_t thread1_id;
  pthread_t thread2_id;

  struct cuenta_parms thread1_args;
  struct cuenta_parms thread2_args;

  thread1_args.nombre = argv[1];
  thread2_args.nombre = argv[1];

  thread1_args.punteroa = cvalores;
  thread2_args.punteroa = -1;

  thread1_args.punterod = 0;
  thread2_args.punterod = cvalores;

  pthread_create (&thread1_id, NULL, &cuenta, &thread1_args);
  pthread_create (&thread2_id, NULL, &cuenta, &thread2_args);

  /*-------INSERTAR AQUÍ-------*/
  pthread_join (thread1_id, NULL);
  pthread_join (thread2_id, NULL);

  printf("La suma total es %d \n",result);
  return 0;
}

Respuestas

![texto alternativo](https://raw.githubusercontent.com/josearangos/thread-api/master/Code/Punto%205/P_5.2.png)

Para medir el tiempo dentro del programa, cualquier zona de código, basta cojer en un punto el "tiempo", y tras hacer las operaciones que quieras, volverlo a medir y restar:
Código

In [0]:

#include <sys/time.h>

struct timeval t, t2;
int microsegundos;
gettimeofday(&t, NULL);

// Codigo ...

gettimeofday(&t2, NULL);
microsegundos = ((t2.tv_usec - t.tv_usec)  + ((t2.tv_sec - t.tv_sec) * 1000000.0f));

microsegundos te devuelve un "tiempo" relativo al comienzo del programa (más o menos). Si lo vuelves a llamar y los restas, obtendrás la diferencia.

Datos obtenidos

![texto alternativo](https://raw.githubusercontent.com/josearangos/thread-api/master/Code/Punto%205/prueba%20tiempo%20P_5.1.png)

![texto alternativo](https://raw.githubusercontent.com/josearangos/thread-api/master/Code/Punto%205/prueba%20de%20tiempo%20P_5.2.png)

Los resultados son los esperados, el segundo programa tarda ams ya que su ejecuciòn depende de la prioridad qu el scheduler de a los hilos necesarios para la ejecuciòn.

6. El profesor de un curso desea un programa en lenguaje C que calcule la desviación estándar (símbolo σ o s) de las notas obtenidas por sus estudiantes en el curso 

![texto alternativo](https://raw.githubusercontent.com/josearangos/thread-api/master/desviacion_estandar.jpg)




**Requisitos**:
* El número de notas es variable (se debe usar memoria dinámica).
* El programa debe crear tantos hilos como se especifique en el parámetro de entrada cantidad_hilos, se debe ejecutar así: 

```./nombre_ejecutable fichero_notas.csv cantidad_hilos```

* Para calcular la desviación estándar, implemente la función: ```calculate_standard_deviation()``` 